In [1]:
#Data Analysis and PreProcessing

In [2]:
import torch
import pandas as pd
from tqdm.notebook import tqdm


In [3]:
df =  pd.read_csv('C:/users/yashm/BERT/amazon_review_polarity_csv/train.csv', names = ['pol','text', 'category'])

In [4]:
df.head()

,pol,text,category
0,2,Stuning even for the non-gamer,This sound track was beautiful! It paints the ...
1,2,The best soundtrack ever to anything.,I'm reading a lot of reviews saying that this ...
2,2,Amazing!,This soundtrack is my favorite music of all ti...
3,2,Excellent Soundtrack,I truly like this soundtrack and I enjoy video...
4,2,"Remember, Pull Your Jaw Off The Floor After He...","If you've played the game, you know how divine..."


In [5]:
df.text.value_counts()

Disappointed                                 9623
Disappointing                                9169
Excellent                                    4367
Great                                        3530
Don't waste your money                       3201
                                             ... 
low carb, high quality, excellent price         1
Late era Larry Masterpiece                      1
Was skeptical at first....but.....              1
Run Away From This Book                         1
Great title - should have left it at that       1
Name: text, Length: 2628753, dtype: int64

In [6]:
df = df[~df.category.str.contains('!')]

In [7]:
df.pol.value_counts()

1    1387142
2    1264275
Name: pol, dtype: int64

In [8]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [9]:
df = pd.read_csv('C:/Users/yashm/BERT/smileannotationsfinal.csv',
                names = ['id', 'text', 'category'])
df.set_index('id', inplace = True)

In [10]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [11]:
df.text.iloc[2]

'@SelectShowcase @Tate_StIves ... Replace with your wish which the artist uses in next installation! It was entralling!'

In [12]:
df.category.value_counts()

nocode               1572
happy                1137
not-relevant          214
angry                  57
surprise               35
sad                    32
happy|surprise         11
happy|sad               9
disgust|angry           7
disgust                 6
sad|disgust             2
sad|angry               2
sad|disgust|angry       1
Name: category, dtype: int64

In [13]:
df = df[~df.category.str.contains('\|')]

In [14]:
df = df[df.category != 'nocode']

In [15]:
df.category.value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [16]:
possible_labels = df.category.unique()

In [17]:
label_dict = {}
for index, possible_label in enumerate(possible_labels):
    label_dict[possible_label] = index

In [18]:
label_dict

{'happy': 0,
 'not-relevant': 1,
 'angry': 2,
 'disgust': 3,
 'sad': 4,
 'surprise': 5}

In [19]:
df['label'] = df.category.replace(label_dict)
df.head()

,text,category,label
id,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0


In [20]:
#Training and Validation Split

In [21]:
from sklearn.model_selection import train_test_split

In [22]:
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size = 0.15,
    random_state = 17,
    stratify = df.label.values
)

In [23]:
df['data_type'] = ['not_set'] *df.shape[0]

In [24]:
df.head()

,text,category,label,data_type
id,,,,
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy,0,not_set
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy,0,not_set
614877582664835073,@Sofabsports thank you for following me back. ...,happy,0,not_set
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy,0,not_set
611570404268883969,@NationalGallery @ThePoldarkian I have always ...,happy,0,not_set


In [25]:
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [26]:
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

In [27]:
#Loading Tokenizer and Encoding our Data

In [29]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [30]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case = True
)

In [37]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df.data_type=='train'].text.values,
    add_special_tokens = True,
    return_attention_mask =True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt'
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df.data_type=='val'].text.values,
    add_special_tokens = True,
    return_attention_mask =True,
    pad_to_max_length = True,
    max_length = 256,
    return_tensors = 'pt'
)

input_ids_train = encoded_data_train['input_ids']
attention_masks_train =  encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df.data_type=='train'].label.values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val =  encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df.data_type=='val'].label.values)

In [38]:
dataset_train = TensorDataset(input_ids_train, attention_masks_train, labels_train)
dataset_val = TensorDataset(input_ids_val, attention_masks_val, labels_val)

In [39]:
len(dataset_train)

1258

In [40]:
len(dataset_val)

223

In [41]:
#importing pretrained bert model

In [42]:
from transformers import BertForSequenceClassification

In [46]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels = len(label_dict),
    output_attentions = False,
    #output_hidden_state=False
)

In [ ]:
#Creating Data Loaders

In [47]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [48]:
batch_size = 4
dataloader_train = DataLoader(
    dataset_train,
    sampler = RandomSampler(dataset_train),
    batch_size = 32
)

#batch_size = 4
dataloader_val = DataLoader(
    dataset_val,
    sampler = RandomSampler(dataset_val),
    batch_size = 32
)

In [52]:
#optimizer and scheduler

In [53]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [56]:
optimizer = AdamW(
    model.parameters(),
    lr=1e-5, # 2e-5 > 5e-5
    eps=1e-8
)

In [58]:
epochs = 10
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs
)

In [59]:
#defining our performance metrics

In [60]:
import numpy as np

In [61]:
from sklearn.metrics import f1_score

In [62]:
#preds = {0.9 0.05 0 0 0}
#preds = {1 0 0 0 0 0}

In [63]:
def f1_score_fun(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return f1_Score(labels_flat, preds_flat, average = 'weighted')

In [65]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true  = labels_flat[labels_flat==label]
        print(f'Class: {label_dict_inversible[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])}/{len(y_true)}')

In [66]:
#creating our training loop

In [67]:
import random
seed_val = 17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [70]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

print(device)


cpu


In [99]:
def evaluate(dataloader_val):
    
    model.era()
    
    loss_val_total = 0
    predictions, true_vals =[], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = { 'input_ids': batch[0],
                  'attention_mask': batch[1],
                  'lables' : batch[2]
        }
        
        with torch.no_grad():
            output = model(**inputs)
        
        loss= outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        
        logits =logits.detach().cpu().numpy()
        label_ids = inputs['.labels'].cput().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
    
    loss_val_avg = loss_val_total/len(dataloader, axis=0)
    true_vals = np.concatenate(true.vals, axis=0)
    
    return loss_val_vag, predictions. true_vals

In [101]:
for epoch in tqdm(range(1, epochs+1)):
    model. train()
    
    loss_train_total = 0
    
    progress_bar = tqdm(dataloader_train,
                       desc = 'Epoch{:1d}'.format(epoch),
                       leave = False,
                        disable = False
                       )
    for batch in progress_bar:
        
        model.zero_grad()
        
        batch =  tuple(b.to(device) for b in batch)
        
        inputs = {'input_ids' : batch[0],'attention_mask' : batch[1],'labels': batch[2]}
        
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({'training_loss':'{:3f}'.format(loss.item()/len(batch))})
    
    torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model')
    
    tqdm.write('\nEpoch {epoch}')
    
    less_train_avg = loss_train_total/len(dataloader)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1.score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')


FileNotFoundError: [Errno 2] No such file or directory: 'Models/BERT_ft_epoch1.model'

In [ ]:
#10 loading and Evaluating our model

In [ ]:
model = BertForSequenceClassification.from_pretrained("bert-base-uncased",
                                                     num_labels=  len(label_dist),
                                                     output_attentions = False,
                                                     output_hidden_states = False)

In [ ]:
model.to(device)
pass

In [ ]:
model.load_stae_dict(
    torch.load('Model/finetuned_bert_epoch_1_gpu_trained.model',
              map_location = torch.device('cpu'))
)

In [ ]:
_, predictions, true_vals = evaluate(dataloader_val)

In [ ]:
accuracy_per_class(prediction, true_vals)